## Imports and Setup

In [2]:
import os
import pandas as pd
import geopandas as gpd
import geemap.foliumap as emap

import sys
sys.path.insert(0, "../utils/")
import gee_utils
import config_utils
import clean_utils

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

%load_ext autoreload
%autoreload 2

## Data Configs

In [5]:
cwd = os.path.dirname(os.getcwd())
config_file = os.path.join(cwd, 'configs/data_config.yaml')
config = config_utils.create_config(config_file)

## Option 1: Download Directly from Data Source 
**Recommended:** Download the GHS-BUILT-C Product directly from the JRC website: https://ghsl.jrc.ec.europa.eu/download.php?ds=builtC

In [11]:
ghsl_zip = os.path.join(cwd, config["rasters_dir"], "ghsl/global/ghsl.zip")
ghsl_folder = os.path.join(cwd, config["rasters_dir"], "ghsl/global")
ghsl_path = os.path.join(cwd, config["rasters_dir"], config["ghsl_file"])
if not os.path.exists(ghsl_path):
    !wget {config["ghsl_url"]} -O {ghsl_zip}
    !unzip {ghsl_zip} -d {ghsl_folder}

--2023-11-20 21:26:39--  https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_BUILT_C_GLOBE_R2023A/GHS_BUILT_C_FUN_E2018_GLOBE_R2023A_54009_10/V1-0/GHS_BUILT_C_FUN_E2018_GLOBE_R2023A_54009_10_V1_0.zip
Resolving jeodpp.jrc.ec.europa.eu (jeodpp.jrc.ec.europa.eu)... 139.191.241.87
Connecting to jeodpp.jrc.ec.europa.eu (jeodpp.jrc.ec.europa.eu)|139.191.241.87|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9505003782 (8.9G) [application/zip]
Saving to: ‘/home/itingzon.unicef/giga/data/rasters/ghsl/global/ghsl.zip’

/home/itingzon.unic 100%[===================>]   8.85G  19.0MB/s    in 11m 58s 

2023-11-20 21:38:37 (12.6 MB/s) - ‘/home/itingzon.unicef/giga/data/rasters/ghsl/global/ghsl.zip’ saved [9505003782/9505003782]

Archive:  /home/itingzon.unicef/giga/data/rasters/ghsl/global/ghsl.zip
  inflating: /home/itingzon.unicef/giga/data/rasters/ghsl/global/GHS_BUILT_C_FUN_E2018_GLOBE_R2023A_54009_10_V1_0.tif  
  inflating: /home/itingzon.unicef/giga/data/rasters

## Option 2: Download from Google Earth Engine (GEE)
Download the GHSL Product via Google Earth Engine. Unless you encounter any problems downloading the global product, we highly recommend using Option 1. 

### Initialize GEE

In [13]:
gee_utils._initialize_gee()

Enter verification code:  4/1AfJohXlgF_zidLAi7S16nkbs0WwSikZPjUh0IaLoC6ASwi34fYuBJYn2lB4



Successfully saved authorization token.


### Export GEE Images to Google Drive

In [15]:
iso_codes = ["BRA", "KIR", "MNG", "NGA", "NAM"]
gee_utils.generate_gee_images(config, iso_codes=iso_codes, folder='GHSL', layer="ghsl")

Processing NAM: 100%|████████████████████| 4/4 [00:12<00:00,  3.13s/it]                                                 


### Merge and Upload GeoTiffs

The `generate_gee_images()` function will export the GEE images to your GDrive folder. Download these images to your local system. Some TIFFs maybe broken down into multiple files; to merge these, simply use the following command:
```sh
conda install gdal
gdalbuildvrt ISO_ghsl.vrt ./*.tif
gdal_translate -of GTiff -co "TILED=YES" ISO_ghsl.vrt ISO_ghsl.tif
```
Once merged, upload the GeoTIFF place it in your GHSL rasters folder under the file name `{iso_code}_ghsl.tiff`, e.g. `data/rasters/ghsl/NAM_ghsl.tiff`. 